In [4]:
# import api keys
from api_keys import *
from IPython.display import JSON
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup as bs

ModuleNotFoundError: No module named 'api_keys'

**City Web Scraping**

In [ ]:
url = ('https://en.wikipedia.org/wiki/List_of_cities_in_the_United_Kingdom')
tables = pd.read_html(url, attrs={'class': 'wikitable sortable'})
type(tables)
#printing number of tables on wiki page
print(len(tables))

3


In [ ]:
# selecting first table on page, as that is the one being referenced
cities= tables[0]
#dropping unnecessary columns
cities= cities.drop(cities.columns[[1,2,3,5]], axis=1)
#renaming columns
cities.rename(columns={'City[3][1]':'City', 'Nation/region':'Country'}, inplace=True)
cities

,City,Country,Population
0,Aberdeen32(Scots: Aiberdeen)(Scottish Gaelic: ...,Scotland,22279338
1,Armagh11(Irish: Ard Mhacha)(Ulster-Scots: Airm...,Northern Ireland,"14,777 (2011)[15]"
2,"Bangor, Wales1",Wales,"18,808 (2011)[16]"
3,Bath1,"South West, England","88,859 (2011)[17]97,311 (urban area, 2010)[18]"
4,Belfast(Irish: Béal Feirste)(Ulster-Scots: Bil...,Northern Ireland,"333,871 (2011)[19]"
...,...,...,...
65,Westminster21,"London, England","219,396 (2011)[21]"
66,Winchester1,"South East, England","116,595 (2011)[21]"
67,Wolverhampton16,"West Midlands, England","249,470 (2011)[21]"
68,Worcester3,"West Midlands, England","98,768 (2011)[21]"


In [ ]:
#cleaning City column
import re
cities['City']= cities['City'].str.replace(r"\(.*|\d.*|\,.*|City of ", "", regex=True)

In [ ]:
#cleaning Country column

cities['Country']=cities['Country'].str.replace(r".*\, ", '', regex=True)

In [ ]:
#adding iso column and City_Country column

cities['ISO'] = pd.Series(["GB" for x in range(len(cities.index))])
cities['City_Country']= cities['City'] + ', ' + cities['ISO']

In [ ]:
#cleaning Population column

cities['Population']=cities['Population'].str.replace(r"\,| .*|\[.*", '', regex=True)
cities.query('City == "Inverness"')

,City,Country,Population,ISO,City_Country
27,Inverness,Scotland,79415,GB,"Inverness, GB"


In [ ]:
cities.query('City == "Stirling"')

,City,Country,Population,ISO,City_Country
58,Stirling,Scotland,3479038,GB,"Stirling, GB"


Merging *cities* with *airports*

In [ ]:
airports= (
pd.read_csv('airports.csv')
    .query('type == "large_airport"')
    .filter(['name','latitude_deg','longitude_deg','iso_country','iso_region','municipality','gps_code','iata_code'])
    .rename(columns={'gps_code':'icao_code'})
    .assign(City_Country = lambda x: x['municipality'] + ', ' + x['iso_country'])
)

In [ ]:
cities_airports= cities.merge(airports, how='inner').drop(['ISO', 'latitude_deg', 'longitude_deg', 'iso_country', 'iso_region', 'municipality', 'iata_code', 'Population', 'City', 'Country'], axis=1)


,City_Country,name,icao_code
0,"Belfast, GB",Belfast International Airport,EGAA
1,"Birmingham, GB",Birmingham International Airport,EGBB
2,"Bristol, GB",Bristol Airport,EGGD
3,"Cardiff, GB",Cardiff International Airport,EGFF
4,"Edinburgh, GB",Edinburgh Airport,EGPH
5,"Exeter, GB",Exeter International Airport,EGTE
6,"Glasgow, GB",Glasgow International Airport,EGPF
7,"Leeds, GB",Leeds Bradford Airport,EGNM
8,"Liverpool, GB",Liverpool John Lennon Airport,EGGP
9,"London, GB",London Luton Airport,EGGW


In [ ]:
cities_airports.rename(columns={'name':'Airport', 'icao_code':'ICAO'}, inplace=True)

,City_Country,Airport,ICAO
0,"Belfast, GB",Belfast International Airport,EGAA
1,"Birmingham, GB",Birmingham International Airport,EGBB
2,"Bristol, GB",Bristol Airport,EGGD
3,"Cardiff, GB",Cardiff International Airport,EGFF
4,"Edinburgh, GB",Edinburgh Airport,EGPH
5,"Exeter, GB",Exeter International Airport,EGTE
6,"Glasgow, GB",Glasgow International Airport,EGPF
7,"Leeds, GB",Leeds Bradford Airport,EGNM
8,"Liverpool, GB",Liverpool John Lennon Airport,EGGP
9,"London, GB",London Luton Airport,EGGW


**Universitiy Data**

In [ ]:
url = ('https://www.4icu.org/gb/a-z/')
web= requests.get(url,'html.parser')
soup = bs(web.content)
#tables = pd.read_html(url)
#type(tables)

#print(len(tables))#printing number of tables on wiki page

**Weather Forcast Data**

In [ ]:
city = "London"
country = "GB"

response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

from IPython.display import JSON
#JSON(response.json())

In [ ]:
forecast_api = response.json()['list']

weather_info = []

for forecast_3h in forecast_api: 
    weather_hour = {}
    # datetime utc
    weather_hour['Date_Time'] = forecast_3h['dt_txt']
    # temperature 
    weather_hour['Temperature'] = forecast_3h['main']['temp']
    # wind
    weather_hour['Wind'] = forecast_3h['wind']['speed']
    # probability precipitation 
    try: weather_hour['Prob_Percipitation'] = float(forecast_3h['pop'])
    except: weather_hour['Prob_Percipitation'] = 0
    # rain
    try: weather_hour['Rain'] = float(forecast_3h['rain']['3h'])
    except: weather_hour['Rain'] = 0
    # wind 
    try: weather_hour['Snow'] = float(forecast_3h['snow']['3h'])
    except: weather_hour['Snow'] = 0
    weather_hour['City_Country'] = city + ', ' + country
    weather_info.append(weather_hour)
    
weather = pd.DataFrame(weather_info)
weather.head()

,Date_Time,Temperature,Wind,Prob_Percipitation,Rain,Snow,City_Country
0,2022-08-10 06:00:00,14.57,2.21,0.0,0,0,"London, GB"
1,2022-08-10 09:00:00,17.48,2.95,0.0,0,0,"London, GB"
2,2022-08-10 12:00:00,23.51,3.52,0.0,0,0,"London, GB"
3,2022-08-10 15:00:00,28.52,3.77,0.0,0,0,"London, GB"
4,2022-08-10 18:00:00,26.73,3.68,0.0,0,0,"London, GB"


**Flight Arrival Data**

In [ ]:
from datetime import datetime, timedelta

airport_icoa = "EGLL"
to_local_time = datetime.now().strftime('%Y-%m-%dT%H:00')
from_local_time = (datetime.now() + timedelta(hours=9)).strftime('%Y-%m-%dT%H:00')
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"


import requests

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flight_api_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

from IPython.display import JSON
#response.json()

NameError: name 'flight_api_key' is not defined

In [ ]:
import pandas as pd
arrivals = pd.json_normalize(response.json()['arrivals'])

arrivals = (arrivals
    .filter(['departure.airport.name','arrival.scheduledTimeLocal','arrival.terminal', 'status', 'number'])
    .assign(ICAO = airport_icoa)
    
)
arrivals.rename(columns={'departure.airport.name':'Departure_City', 'arrival.scheduledTimeLocal': 'Arrival_Time', 'arrival.terminal': 'Terminal', 'status': 'Status', 'number': 'Flight'}, inplace=True)

**Check tables**

In [ ]:
cities.head()

,City,Country,Population,ISO,City_Country
0,Aberdeen,Scotland,22279338,GB,"Aberdeen, GB"
1,Armagh,Northern Ireland,14777,GB,"Armagh, GB"
2,Bangor,Wales,18808,GB,"Bangor, GB"
3,Bath,England,88859,GB,"Bath, GB"
4,Belfast,Northern Ireland,333871,GB,"Belfast, GB"


In [ ]:
cities_airports.head()

,City_Country,Airport,ICAO
0,"Belfast, GB",Belfast International Airport,EGAA
1,"Birmingham, GB",Birmingham International Airport,EGBB
2,"Bristol, GB",Bristol Airport,EGGD
3,"Cardiff, GB",Cardiff International Airport,EGFF
4,"Edinburgh, GB",Edinburgh Airport,EGPH


In [ ]:
weather.head()

,Date_Time,Temperature,Wind,Prob_Percipitation,Rain,Snow,City_Country
0,2022-08-10 06:00:00,14.57,2.21,0.0,0,0,"London, GB"
1,2022-08-10 09:00:00,17.48,2.95,0.0,0,0,"London, GB"
2,2022-08-10 12:00:00,23.51,3.52,0.0,0,0,"London, GB"
3,2022-08-10 15:00:00,28.52,3.77,0.0,0,0,"London, GB"
4,2022-08-10 18:00:00,26.73,3.68,0.0,0,0,"London, GB"


In [ ]:
arrivals.head()

NameError: name 'arrivals' is not defined

**Connect Python to MySQL**

In [ ]:
schema="gans"
host="data-eng-project.crq4hzbbr6f3.eu-west-2.rds.amazonaws.com"
user="admin"
password="Kjmqtpie96!"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cities.to_sql('cities', if_exists='append', con=con, index=False)
cities_airports.to_sql('cities_airports', if_exists='append', con=con, index=False)
weather.assign(Date_Time = lambda x: pd.to_datetime(x['Date_Time'])).to_sql('weather', if_exists='append', con=con, index=False)
arrivals.to_sql('arrivals', if_exists='append', con=con, index=False)